Start by importing packages - gcsfs is specifically for reading files from google cloud file storage

In [1]:
import gcsfs
import xarray as xr

Ignore the below cell for now; I'm having trouble connecting to the daskcluster

In [12]:
'''
from dask.distributed import Client
from dask_gateway import GatewayCluster

cluster = GatewayCluster()
client = Client(cluster)
cluster.scale(2)
'''

OSError: Timed out trying to connect to gateway://traefik-ohw-dask-gateway.ohw:80/ohw.fb4e6056927347cea6300acd3d6e27e6 after 30 s

Telling jupyter how to access the Google Cloud Storage file system

In [2]:
fs = gcsfs.GCSFileSystem()

In [3]:
mapper = fs.get_mapper("gs://ohw23-oil/S1A_IW_GRDH_1SDV_2016b.zarr/")

Import both datasets! consolidated=True tells xarray to read the dataset with consolidated metadata (much faster!)

In [4]:
s1a2016 = xr.open_zarr(mapper,consolidated=True)

In [5]:
mapper2 = fs.get_mapper("gs://ohw23-oil/S1A_IW_GRDH_1SDV_2017b.zarr/")

In [6]:
s1a2017 = xr.open_zarr(mapper2,consolidated=True)

In [7]:
s1a2016

<xarray.Dataset>
Dimensions:                      (lat: 21313, lon: 33199)
Coordinates:
  * lat                          (lat) float64 32.37 32.37 32.37 ... 30.45 30.45
  * lon                          (lon) float64 31.77 31.77 31.77 ... 34.75 34.75
Data variables:
    Sigma0_VV                    (lat, lon) float32 dask.array<chunksize=(667, 1038), meta=np.ndarray>
    crs                          int32 ...
    incidenceAngleFromEllipsoid  (lat, lon) float32 dask.array<chunksize=(667, 1038), meta=np.ndarray>
    metadata                     int8 ...
Attributes:
    Conventions:       CF-1.4
    TileSize:          292:400
    metadata_profile:  beam
    metadata_version:  0.5
    product_type:      GRD
    start_date:        06-AUG-2016 15:48:30.031330
    stop_date:         06-AUG-2016 15:48:55.029747
    title:             Sentinel-1 IW Level-1 GRD Product

Pull out data as array (optional!) and check chunksizes

In [11]:
s1a2016_arr = s1a2016['Sigma0_VV']

In [12]:
s1a2016_arr.chunksizes

Frozen({'lat': (667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 667, 636), 'lon': (1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1021)})

show min, max, mean values as data check

In [17]:
print(s1a2016_arr.min().compute())
print(s1a2016_arr.max().compute())
print(s1a2016_arr.mean().compute())

<xarray.DataArray 'Sigma0_VV' ()>
array(1.0632014e-10, dtype=float32)
<xarray.DataArray 'Sigma0_VV' ()>
array(1129.1149, dtype=float32)
<xarray.DataArray 'Sigma0_VV' ()>
array(0.02132743, dtype=float32)


Filter out data by threshold

In [18]:
threshold = 0.05
s1a2016_arr_filt = s1a2016_arr.where(s1a2016_arr < threshold)